<a href="https://colab.research.google.com/github/jldz9/InSARScript/blob/tutorial/InSARScript_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# InSARScript Tutorial
Welcome to InSARScript! InSAR Script is an open-source package designed to support the full InSAR processing pipeline. The primary goal of this package is to provide a streamlined and user-friendly InSAR processing experience across multiple satellite products.

This tutorial will guide you through the essential steps of using InSARScript

## Setup Environment

This step will setup the program environment, this may take several minutes. Runtime restart might be required after installation.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!git clone https://github.com/jldz9/InSARScript.git
!pip install -e /content/InSARScript



## Account Setup
InSARScript requires multiple online accounts for full functionality. Registration for these accounts is completely free.

Register [NASA Earthdata](https://urs.earthdata.nasa.gov/)

Register [Copernicus Climate Data Store](https://cds.climate.copernicus.eu/)

This account is required for searching satellite scenes, downloading DEMs and orbit files, and submitting online interferogram processing jobs.
Once you've register your account please put your corresponding info below:

`  ⚠️ Note: This tutorial will not store or share your personal information — your credentials are used only for demonstration purposes during this session.`

In [37]:
Earthdata_Username = 'Your_Earthdata_Username'
Earthdata_Password = 'Your_Earthdata_Password'

Once you registered Copernicus Climate Data Store account, Click [Here](https://cds.climate.copernicus.eu/how-to-api) to get your API token.


In [38]:
CCDS_API_Key = 'Your_CCDS_API_Token'

In [39]:
from pathlib import Path
import os
netrc_path = Path.home() / ".netrc"
# Create netrc file
rc_path = Path.home() / ".cdsapirc"
# Write credentials to ~/.netrc
with open(netrc_path, "w") as f:
    f.write(f"""machine urs.earthdata.nasa.gov
login {Earthdata_Username}
password {Earthdata_Password}
""")

with open(rc_path, "w") as f:
    f.write(f"""url: https://cds.climate.copernicus.eu/api
key: {CCDS_API_Key}
""")


## Search

InSARSCript currently support searching of Sentinel 1 SLC. To define AOI, simply create a list of coordinates by ***[min_long, min_lat, max_long, max_lat]***


In [ ]:
from insarscript import S1_SLC
AOI = [-113.353, 37.546, -112.10, 38.415]
s1 = S1_SLC(
    platform=['Sentinel-1A', 'Sentinel-1B', 'Sentinel-1C'],
    AscendingflightDirection=False,
    bbox = AOI,
    start='2020-06-01',
    end='2020-09-30',
    output_dir = '~/tmp',
)
results = s1.search()

 your AOI probably spans multiple scenes. To filter the search results, you can use:

In [ ]:
s1.footprint()

This will display a footprint map of the available Sentinel-1 scenes that covers the AOI. The stack indicates numbers of SAR sences in that footprint

OR

In [ ]:
s1.summary()

This will output the summary of availiable Sentinel-1 scenes that covers the AOI.

if we decide to choose one SAR sence stacks (Path:100 Frame:466) for further process , simply do:

In [32]:
filter_results = s1.pick((100,466))

You can then review the footprint and summary to confirm that the filter has been applied successfully.

In [ ]:
s1.footprint()
s1.summary()

## Interferogram Process

After locating SAR scene stacks that cover the Area of Interest (AOI), the next step is to generate unwrapped interferograms in preparation for the SBAS processing. This tutorial use HyP3 online service as interferogram processor.

Before submitting jobs to HyP3 to generate interferograms, we first need to prepare the list of connection pair using `select_pairs`

In [ ]:
from insarscript import select_pairs
sbas_pairs = select_pairs(search_results=filter_results[(100,466)], dt_targets=(6, 12, 24, 36, 48),  force_connect=False)

we then submit our interferogram pairs to Hyp3 server for interferogram processing

In [ ]:
from insarscript import Hyp3_InSAR_Processor

hyp3_insar = Hyp3_InSAR_Processor(
    pairs = sbas_pairs,
    out_dir = "/content/"
)
batch = hyp3_insar.submit()

You may check the status of current job by using `.refresh()`

In [ ]:
batch = hyp3_insar.refresh()

Processing will take roughly 30 mins for every 100 interferograms depends on the ASF server load, you can save the job script to check back later by: `.save()`

In [ ]:
save_path = hyp3_insar.save('/content/drive/MyDrive/hyp3_job_p100f466.json')

You can load back the job later and check the job status.

In [ ]:
from insarscript import Hyp3_InSAR_Processor
insar_job_reload = Hyp3_InSAR_Processor.load('/content/drive/MyDrive/hyp3_job_p100f466.json')
batch = insar_job_reload.refresh()

Once all jobs are completed, use `.download()` to download processed interferograms

In [ ]:
insar_job_reload.download(output='/content/')

## SBAS

Running SBAS requires a set of interferograms generated from SAR image pairs with small spatial and temporal baselines. This approach is designed to minimize phase decorrelation and improve the quality of the deformation signal.

InSARScript utilizes MintPy, a powerful open-source toolbox for InSAR time-series analysis, to perform the SBAS workflow.

This would take roughtly 1 hr to run in google colab environment

In [ ]:
from insarscript import Hyp3_SBAS
sbas_run = Hyp3_SBAS(
    hyp3_dir = '/content/'
)
sbas_run.prep_data()
sbas_run.run()

To view the mean velocity map

In [ ]:
import rasterio
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from insarscript.utils import h5_to_raster

h5_to_raster('/content/velocity.h5')
raster_path = '/content/velocity_velocity.tif'
with rasterio.open(raster_path) as src:
    data = src.read(1)
nodata_value = -9999
data[data == nodata_value] = np.nan
plt.figure(figsize=(6, 6))
plt.imshow(data)
plt.colorbar(label="Value")
plt.title("Raster preview")
plt.axis("off")
plt.show()